In [1]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account

In [2]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))

load_dotenv

mnemonic = os.getenv('MNEMONIC', "win %%%%%%% rely void arch")
print(mnemonic)

X


In [3]:

def derive_wallets(mnemonic, coin, numderive):
 
    command = f'php derive -g --mnemonic="{mnemonic}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
   
    keys = json.loads(output)
    return  keys

In [4]:

coins = {"eth", "btc-test", "btc"}
numderive = 3

In [5]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [6]:

eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']

In [37]:
print("eth_PrivateKey:",json.dumps(eth_PrivateKey, indent=1))
print("btc_PrivateKey:",json.dumps(btc_PrivateKey, indent=1))
print(json.dumps(keys, indent=1))

eth_PrivateKey: "0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc"
btc_PrivateKey: "cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt"
{
 "btc-test": [
  {
   "path": "m/44'/1'/0'/0/0",
   "address": "mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD",
   "xprv": "tprv8khhNkzGYHzyhzgCwAKEf4xVvkBbV2vWqBQpybEMhGHivv3WgwhnjdCguDwQ1ujRnN5JWEM8q8nE8zjHSrthSTvMcxb6V575s5x2zT3q1Pk",
   "xpub": "tpubDHPjXB2WgfgebThzpoyq4UccVmhXeN7RQV1cG7Gf7Y67mQJHKLXNv7pZ5M2WvzFPpQyVgMB9znNEFMefiFvgn7r72wMott5xBJksEZUfWX5",
   "privkey": "cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt",
   "pubkey": "02dcb7a11c7172966e58679c0cb207bfb95d119cc55f9fb318d0a522aaff5b9577",
   "pubkeyhash": "3b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb8",
   "index": 0
  },
  {
   "path": "m/44'/1'/0'/0/1",
   "address": "mmUQuAvBYqVpKzayhaYvnJe9VuwkRdSWPu",
   "xprv": "tprv8khhNkzGYHzyj6e6K8iKuPTCw6YLNRA3tRKCphDT4Yis5AKoPbwFLH7zqay3WqqcsnHCwJZedosPPAXcrHehSKV7MBy57KVPQxyQ3tHyq4g",
   "xpub": "tpubDHPjXB2WgfgecZftCnNvJo7KW84G

In [28]:
def priv_key_to_account(coin, priv_key):

    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

print("eth_acc = ",eth_acc)
print("btc_acc=", btc_acc)

eth_acc =  <eth_account.signers.local.LocalAccount object at 0x00000261D65BF250>
btc_acc= <PrivateKeyTestnet: mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD>


In [9]:
def create_trx(coin, account, recipient, amount):
    
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])  


In [10]:
def send_trx(coin, account, recipient, amount):
    
    if coin == "eth": 
        trx_eth = create_trx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_trx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

In [11]:
create_trx(BTCTEST,btc_acc,"mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD", 0.01)

'{"unspents":[{"amount":1000000,"confirmations":15,"script":"76a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac","txid":"872cd6137d5a2ac92c1121cfb372fec5605628f324da2504e1408da93647c5dc","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":210315,"confirmations":15,"script":"76a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac","txid":"872cd6137d5a2ac92c1121cfb372fec5605628f324da2504e1408da93647c5dc","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD",1000000],["mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD",172167]]}'

In [12]:
send_trx(BTCTEST,btc_acc,"mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD", 0.01)

'0100000002dcc54736a98d40e10425da24f3285660c5fe72b3cf21112cc92a5a7d13d62c87000000006a47304402201e429edb6e797326123f47fda4ad691279f024780b63f6e10b2f9f890d4eb4ed02200a3fd9978509d1e01f06a5e81f6e5e7f2422d238f368ff4897d270cfa5e7330f012102dcb7a11c7172966e58679c0cb207bfb95d119cc55f9fb318d0a522aaff5b9577ffffffffdcc54736a98d40e10425da24f3285660c5fe72b3cf21112cc92a5a7d13d62c87010000006b483045022100f7b242ec3c45c2fa7c403783d4d3abb7051d3606a5850bc99b094631e21287a90220382dcce2767b8ef0f3f9e04c28d41f53c9e71b6f5f64905840855ee92f4e6904012102dcb7a11c7172966e58679c0cb207bfb95d119cc55f9fb318d0a522aaff5b9577ffffffff0240420f00000000001976a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac87a00200000000001976a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac00000000'

In [13]:
##ETH

In [14]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [15]:
from web3 import Web3, HTTPProvider

In [16]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [17]:
w3.isConnected()

True

In [30]:
w3.eth.getBalance("0xaA9FB096fE1B2BdD8e96AB6670fcFE447f5BeC83")


100000000000000000000

In [19]:
create_trx(ETH,eth_acc,"0xA0A0173f7DdFee91df5C052319F20Aad81312a6B", 1000)

{'to': '0xA0A0173f7DdFee91df5C052319F20Aad81312a6B',
 'from': '0x02194a55DDA1B029F70220D53f099F38c629ab36',
 'value': 1000,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}

In [20]:
send_trx(ETH, eth_acc,"0xA0A0173f7DdFee91df5C052319F20Aad81312a6B", 1000)

ValueError: {'message': "sender doesn't have enough funds to send tx. The upfront cost is: 420000000001000 and the sender's account only has: 0", 'code': -32000, 'data': {'stack': "Error: sender doesn't have enough funds to send tx. The upfront cost is: 420000000001000 and the sender's account only has: 0\n    at VM.<anonymous> (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\lib\\runTx.ts:114:11)\n    at step (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\dist\\runTx.js:33:23)\n    at Object.next (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\dist\\runTx.js:14:53)\n    at fulfilled (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\res